In [1]:
from numpy import *
import pandas as pd
import ethnicolr
import platform

from IPython.core.display import display, HTML

# choose root directory for data files
if platform.system() == 'Darwin':
    data_dir = '/Users/Ethan/Documents/Finances and records/2020-06-30 - Helmet Law Working Group/Data/'
elif platform.system() == 'Linux':
    data_dir = '/dat1/ethancc/CSG/'

ModuleNotFoundError: No module named 'ethnicolr'

In [189]:
names = pd.read_csv(data_dir + 'seattle_municipal_court_batch2_names.csv')

# help(ethnicolr.pred_fl_reg_name)
# help(ethnicolr.pred_nc_reg_name)
# help(ethnicolr.pred_census_ln)
# help(ethnicolr.census_ln)

last_name_census_match = ethnicolr.census_ln(names.copy(),namecol='Defendant Last Name',year=2010)
last_name_census_pred = ethnicolr.pred_census_ln(names.copy(),namecol='Defendant Last Name',year=2010)
full_name_NC_pred = ethnicolr.pred_nc_reg_name(names.copy(),lname_col='Defendant Last Name',
                                               fname_col='Defendant First Name')
full_name_FL_pred = ethnicolr.pred_fl_reg_name(names.copy(),lname_col='Defendant Last Name',
                                               fname_col='Defendant First Name')

In [190]:
last_name_census_match[last_name_census_match == '(S)'] = NaN

last_name_census_match[['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic']] = \
    last_name_census_match[['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic']].astype(float)

# fill rows with some (S) values, indicating partially missing Census matches for certain race/ethnicity categories
# ... divide remaining percent evenly between (S) placeholders 
num_missing = last_name_census_match[['pctwhite','pctblack','pctapi',
                                      'pctaian','pct2prace','pcthispanic']].apply(lambda x: sum(isnan(x)),axis=1)
low_match_count_mask = where(logical_and(num_missing > 0, num_missing < 6))[0]
rows_to_be_filled = last_name_census_match.loc[low_match_count_mask][['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic']]
fill_values = (100 - rows_to_be_filled.sum(axis=1)) / sum(rows_to_be_filled.isna(),axis=1)
fill_values_all = pd.DataFrame(index=last_name_census_match.index,data=full((len(last_name_census_match),6),NaN),
                               columns=['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic'])
for colname in ['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic']:
    fill_values_all[colname] = fill_values
last_name_census_match = last_name_census_match.fillna(value=fill_values_all)

In [191]:
last_name_census_match.to_csv(data_dir + 'seattle_municipal_court_batch2_names_last_name_census_match.csv')
last_name_census_pred.to_csv(data_dir + 'seattle_municipal_court_batch2_names_last_name_census_pred.csv')
full_name_NC_pred.to_csv(data_dir + 'seattle_municipal_court_batch2_names_full_name_NC_pred.csv')
full_name_FL_pred.to_csv(data_dir + 'seattle_municipal_court_batch2_names_full_name_FL_pred.csv')